In [2]:
import numpy as np
from sklearn.model_selection import KFold
import cv2
import os
import keras
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model, model_from_json
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import h5py


Using TensorFlow backend.
/home/utkarsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/utkarsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/utkarsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/utkarsh/anaconda3/lib/python3.7/site-packages/t

In [5]:
def load_dataset(folder):
    X=[]
    Y=[]
    num = [i for i in range(26)]
    encoding = to_categorical(num)
    for subfolder in os.listdir(folder):
        label = subfolder[0]
        index = ord(label) - ord('A')
        for data in os.listdir(os.path.join(folder,subfolder)):
            X.append(cv2.imread(os.path.join(folder,subfolder,data),cv2.IMREAD_GRAYSCALE))
            Y.append(encoding[index])
    X=np.array(X)
    Y=np.array(Y)
    return X,Y
            

In [6]:
def DeCaptcha(input_shape, classes=26):
    
    X_input = Input(input_shape)
    X = Conv2D(5,(3,3),strides=(1,1),padding='valid',name='conv1')(X_input)
    X = BatchNormalization(axis = -1, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2),strides=(2,2))(X)
    
    X = Conv2D(10,(3,3),strides=(1,1),padding='same',name='conv2')(X)
    X = BatchNormalization(axis = -1, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2),strides=(2,2))(X)
    
    X = Conv2D(15,(3,3),strides=(2,2),padding='same',name='conv3')(X)
    X = BatchNormalization(axis = -1, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2),strides=(2,2))(X)
    
    X = Flatten()(X)
    X = Dense(256, activation='relu', name='dense')(X)
    X = Dense(classes, activation='softmax', name='output')(X)
    
    model=Model(inputs = X_input, outputs=X, name='DeCaptcha')
    return model
    
    

In [9]:
X,Y = load_dataset('processed_data')
X=X/255
(m,n_C,n_W)= X.shape
X=X.reshape((m,n_C,n_W,1))
model= DeCaptcha((n_C,n_W,1), 26)
adam= keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_path = "../cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cb =  ModelCheckpoint(filepath = checkpoint_path, save_weights_only=True, verbose=1)
model.fit(X, Y, epochs=4, batch_size=32, callbacks = [cb])

Epoch 1/4
7008/7008 [==============================] - 61s 9ms/step - loss: 0.3110 - accuracy: 0.9285

Epoch 00001: saving model to ../cp.ckpt
Epoch 2/4
7008/7008 [==============================] - 64s 9ms/step - loss: 0.0023 - accuracy: 1.0000

Epoch 00002: saving model to ../cp.ckpt
Epoch 3/4
7008/7008 [==============================] - 64s 9ms/step - loss: 9.6460e-04 - accuracy: 1.0000

Epoch 00003: saving model to ../cp.ckpt
Epoch 4/4
7008/7008 [==============================] - 63s 9ms/step - loss: 5.3436e-04 - accuracy: 1.0000

Epoch 00004: saving model to ../cp.ckpt


In [10]:
model_json = model.to_json()
with open("model_num.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_num.h5")

 
